In [8]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai_api_key = os.getenv("OPENAI_API_KEY")

In [9]:
openai.api_key = openai_api_key
client = openai.Client()

In [44]:
mensagens = ""

mensagens = [{'role' : 'user', 'content' : 'O que é uma maça em 5 palavras?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

In [45]:
print(resposta.choices[0].message.content)

Fruta redonda e saborosa.


In [46]:
mensagens.append({'role': 'assistant', 'content': resposta.choices[0].message.content})

In [47]:
mensagens.append({'role': 'user', 'content': 'E qual é a sua cor?'})

In [48]:
mensagens

[{'role': 'user', 'content': 'O que é uma maça em 5 palavras?'},
 {'role': 'assistant', 'content': 'Fruta redonda e saborosa.'},
 {'role': 'user', 'content': 'E qual é a sua cor?'}]

In [49]:
resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

Vermelha, verde ou amarela.
